# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Week 2: Unsupervised NLP</font>

# <font color="#003660">Notebook 3: Classification with Word Embeddings</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you will be able to...</b><br><br>
        ... train a classifier with mean word embeddings as features.
    </font>
</div>
</center>
</p>

# Import packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `numpy` is a library adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `SQLAlchemy`, together with `pymysql`, allows to communicate with SQL databases.
- `getpass` provides function to safely enter passwords.
- `spacy` offers industrial-strength natural language processing.
- `en_core_web_md` is a pre-trained Spacy model that has word embeddings included
- `sklearn` is the de-facto standard machine learning package in Python.

In [ ]:
# Install packages
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import getpass
import spacy
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load documents

We load our data from a MySQL database. For security reasons, we don't store the database credentials here; please have a look at Panda to get them.

In [ ]:
# Get credentials
user = input("Username: ")
passwd = getpass.getpass("Password: ")
server = input("Server: ")
db = input("Database: ")

# Create an engine instance (SQLAlchemy)
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(user, passwd ,server, db))

# Define SQL query
sql_query = "SELECT * FROM WineDataset"

# Query dataset (pandas)
corpus = pd.read_sql(sql=sql_query, con=engine)

Username: student
Password: ··········
Server: manila.uni-paderborn.de
Database: aml4ta


In [ ]:
corpus.shape

(129971, 16)

# Preprocess documents

Split data into training, validation, and test set.

In [ ]:
training = corpus[corpus["testset"] == 0]
validation = training.iloc[80000:100000,]
training = training.iloc[0:80000,]
test = corpus[corpus["testset"] == 1]

In [ ]:
print(training.shape)
print(validation.shape)
print(test.shape)

(80000, 16)
(20000, 16)
(29970, 16)


# Vectorize documents

Instead of using a BoW model, we will vectorize the documents by computing the average word embeddings over all words of a document. For this, we will use the word embeddings we have trained on the wine dataset.

Convert the trained word embeddings to a format that Spacy understands (read more: https://spacy.io/api/cli#init-vectors).

In [ ]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy init vectors en /content/drive/MyDrive/colab_notebooks/AML4TA2022/Session_03/data/wine_300dim_10minwords_4context /content/drive/MyDrive/colab_notebooks/AML4TA2022/Session_03/data

Load the custom embeddings and extract the mean word embeddings for each document.

In [ ]:
nlp = spacy.load("/content/drive/MyDrive/colab_notebooks/AML4TA2022/Session_03/data")

def spacy_prep(dataset):
  vectors = []
  dataset = dataset.to_dict("records")
  for i, entry in enumerate(dataset):
      text = nlp(entry[u'description'])
      vectors.append(text.vector)
  return(np.array(vectors))

In [ ]:
X_training = spacy_prep(training)

In [ ]:
X_training.shape

(80000, 300)

Store the labels that we want to predict in a separate variable.

In [ ]:
y_training = training["verygood"]
y_training.describe()

count    80000.000000
mean         0.095137
std          0.293407
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: verygood, dtype: float64

# Train classifier on training set

Fit a classifier with word embeddings as the features and wine quality (i.e., `verygood` variable) as the label.

In [ ]:
clf = LogisticRegression(max_iter=1000).fit(X_training, y_training)

# Evaluate accuracy on validation set

Before trying to predict the labels for the official test set (on Kaggle), we evaluate the predictive accurcay of our model on the validation set.

In [ ]:
X_validation = spacy_prep(validation)

In [ ]:
y_validation = validation["verygood"]

Call the predict function of our model with the validation data and calculate precision, recall and F1-score.

In [ ]:
predictions_validation = clf.predict(X_validation)
print(metrics.classification_report(y_validation, predictions_validation))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95     18001
           1       0.59      0.18      0.28      1999

    accuracy                           0.91     20000
   macro avg       0.75      0.59      0.62     20000
weighted avg       0.88      0.91      0.88     20000

